RNN을 이용한 텍스트 생성(Text Generation using RNN) [Reference](https://wikidocs.net/45101)

In [1]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [2]:
text = """경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다\n"""

In [3]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
vocab_size = len(tokenizer.word_index) + 1
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 12


In [4]:
print(tokenizer.word_index)

{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}


In [5]:
sequences = list()
for line in text.split('\n'): # 줄바꿈 문자를 기준으로 문장 토큰화
    encoded = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

print('학습에 사용할 샘플의 개수: %d' % len(sequences))

학습에 사용할 샘플의 개수: 11


In [6]:
print(sequences)

[[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9], [8, 1, 9, 10], [8, 1, 9, 10, 1], [8, 1, 9, 10, 1, 11]]


In [7]:
max_len = max(len(l) for l in sequences) # 모든 샘플에서 길이가 가장 긴 샘플의 길이 출력
print('샘플의 최대 길이 : {}'.format(max_len))

샘플의 최대 길이 : 6


In [8]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')

In [9]:
print(sequences)

[[ 0  0  0  0  2  3]
 [ 0  0  0  2  3  1]
 [ 0  0  2  3  1  4]
 [ 0  2  3  1  4  5]
 [ 0  0  0  0  6  1]
 [ 0  0  0  6  1  7]
 [ 0  0  0  0  8  1]
 [ 0  0  0  8  1  9]
 [ 0  0  8  1  9 10]
 [ 0  8  1  9 10  1]
 [ 8  1  9 10  1 11]]


In [10]:
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]

In [11]:
print(X)

[[ 0  0  0  0  2]
 [ 0  0  0  2  3]
 [ 0  0  2  3  1]
 [ 0  2  3  1  4]
 [ 0  0  0  0  6]
 [ 0  0  0  6  1]
 [ 0  0  0  0  8]
 [ 0  0  0  8  1]
 [ 0  0  8  1  9]
 [ 0  8  1  9 10]
 [ 8  1  9 10  1]]


In [12]:
print(y)

[ 3  1  4  5  1  7  1  9 10  1 11]


In [13]:
y = to_categorical(y, num_classes=vocab_size)

In [14]:
print(y)

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

In [16]:
embedding_dim = 10
hidden_units = 32

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(SimpleRNN(hidden_units))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)

Epoch 1/200
1/1 - 1s - loss: 2.5017 - accuracy: 0.0909 - 851ms/epoch - 851ms/step
Epoch 2/200
1/1 - 0s - loss: 2.4882 - accuracy: 0.1818 - 2ms/epoch - 2ms/step
Epoch 3/200
1/1 - 0s - loss: 2.4745 - accuracy: 0.1818 - 2ms/epoch - 2ms/step
Epoch 4/200
1/1 - 0s - loss: 2.4605 - accuracy: 0.2727 - 2ms/epoch - 2ms/step
Epoch 5/200
1/1 - 0s - loss: 2.4462 - accuracy: 0.2727 - 2ms/epoch - 2ms/step
Epoch 6/200
1/1 - 0s - loss: 2.4314 - accuracy: 0.3636 - 2ms/epoch - 2ms/step
Epoch 7/200
1/1 - 0s - loss: 2.4161 - accuracy: 0.3636 - 2ms/epoch - 2ms/step
Epoch 8/200
1/1 - 0s - loss: 2.4002 - accuracy: 0.3636 - 2ms/epoch - 2ms/step
Epoch 9/200
1/1 - 0s - loss: 2.3835 - accuracy: 0.3636 - 4ms/epoch - 4ms/step
Epoch 10/200
1/1 - 0s - loss: 2.3661 - accuracy: 0.3636 - 2ms/epoch - 2ms/step
Epoch 11/200
1/1 - 0s - loss: 2.3479 - accuracy: 0.3636 - 2ms/epoch - 2ms/step
Epoch 12/200
1/1 - 0s - loss: 2.3287 - accuracy: 0.3636 - 3ms/epoch - 3ms/step
Epoch 13/200
1/1 - 0s - loss: 2.3087 - accuracy: 0.3636 -

Epoch 104/200
1/1 - 0s - loss: 0.7748 - accuracy: 0.8182 - 997us/epoch - 997us/step
Epoch 105/200
1/1 - 0s - loss: 0.7616 - accuracy: 0.8182 - 2ms/epoch - 2ms/step
Epoch 106/200
1/1 - 0s - loss: 0.7485 - accuracy: 0.8182 - 2ms/epoch - 2ms/step
Epoch 107/200
1/1 - 0s - loss: 0.7357 - accuracy: 0.8182 - 2ms/epoch - 2ms/step
Epoch 108/200
1/1 - 0s - loss: 0.7230 - accuracy: 0.8182 - 998us/epoch - 998us/step
Epoch 109/200
1/1 - 0s - loss: 0.7106 - accuracy: 0.8182 - 996us/epoch - 996us/step
Epoch 110/200
1/1 - 0s - loss: 0.6984 - accuracy: 0.9091 - 997us/epoch - 997us/step
Epoch 111/200
1/1 - 0s - loss: 0.6863 - accuracy: 0.9091 - 2ms/epoch - 2ms/step
Epoch 112/200
1/1 - 0s - loss: 0.6745 - accuracy: 0.9091 - 2ms/epoch - 2ms/step
Epoch 113/200
1/1 - 0s - loss: 0.6629 - accuracy: 0.9091 - 2ms/epoch - 2ms/step
Epoch 114/200
1/1 - 0s - loss: 0.6515 - accuracy: 0.9091 - 997us/epoch - 997us/step
Epoch 115/200
1/1 - 0s - loss: 0.6402 - accuracy: 0.9091 - 2ms/epoch - 2ms/step
Epoch 116/200
1/1 - 

In [17]:
def sentence_generation(model, tokenizer, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word
    sentence = ''

    # n번 반복
    for _ in range(n):
        # 현재 단어에 대한 정수 인코딩과 패딩
        encoded = tokenizer.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=5, padding='pre')
        # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
        result = model.predict(encoded, verbose=0)
        result = np.argmax(result, axis=1)

        for word, index in tokenizer.word_index.items(): 
            # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면 break
            if index == result:
                break

        # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        current_word = current_word + ' '  + word

        # 예측 단어를 문장에 저장
        sentence = sentence + ' ' + word

    sentence = init_word + sentence
    return sentence

In [18]:
print(sentence_generation(model, tokenizer, '경마장에', 4))

경마장에 있는 말이 뛰고 있다


In [19]:
print(sentence_generation(model, tokenizer, '그의', 2))

그의 말이 법이다


In [20]:
print(sentence_generation(model, tokenizer, '가는', 5))

가는 말이 고와야 오는 말이 곱다


In [21]:
import pandas as pd
import numpy as np
from string import punctuation

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [22]:
df = pd.read_csv('ArticlesApril2018.csv')
df.head()

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleaders’ Settlement Offer: ...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"“I understand that they could meet with us, pa...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,What’s it like to eat at the second incarnatio...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...
3,5adf40d2068401528a2aa619,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...
4,5adf3d64068401528a2aa60f,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Toronto’s Ri...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...


In [23]:
print('열의 개수: ',len(df.columns))
print(df.columns)

열의 개수:  15
Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


In [24]:
print(df['headline'].isnull().values.any())

False


In [25]:
headline = []
# 헤드라인의 값들을 리스트로 저장
headline.extend(list(df.headline.values)) 
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

In [26]:
print('총 샘플의 개수 : {}'.format(len(headline)))

총 샘플의 개수 : 1324


In [27]:
headline = [word for word in headline if word != "Unknown"]
print('노이즈값 제거 후 샘플의 개수 : {}'.format(len(headline)))

노이즈값 제거 후 샘플의 개수 : 1214


In [28]:
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?']

In [29]:
def repreprocessing(raw_sentence):
    preproceseed_sentence = raw_sentence.encode("utf8").decode("ascii",'ignore')
    # 구두점 제거와 동시에 소문자화
    return ''.join(word for word in preproceseed_sentence if word not in punctuation).lower()

preprocessed_headline = [repreprocessing(x) for x in headline]
preprocessed_headline[:5]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression']

In [30]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(preprocessed_headline)
vocab_size = len(tokenizer.word_index) + 1
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 3494


In [31]:
sequences = list()

for sentence in preprocessed_headline:

    # 각 샘플에 대한 정수 인코딩
    encoded = tokenizer.texts_to_sequences([sentence])[0] 
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

sequences[:11]

[[99, 269],
 [99, 269, 371],
 [99, 269, 371, 1115],
 [99, 269, 371, 1115, 582],
 [99, 269, 371, 1115, 582, 52],
 [99, 269, 371, 1115, 582, 52, 7],
 [99, 269, 371, 1115, 582, 52, 7, 2],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116],
 [100, 3]]

In [32]:
index_to_word = {}
for key, value in tokenizer.word_index.items(): # 인덱스를 단어로 바꾸기 위해 index_to_word를 생성
    index_to_word[value] = key

print('빈도수 상위 582번 단어 : {}'.format(index_to_word[582]))

빈도수 상위 582번 단어 : offer


In [33]:
max_len = max(len(l) for l in sequences)
print('샘플의 최대 길이 : {}'.format(max_len))

샘플의 최대 길이 : 24


In [34]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences[:3])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0   99  269]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   99  269  371]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0   99  269  371 1115]]


In [35]:
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]

In [36]:
print(X[:3])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0  99]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0  99 269]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0  99 269 371]]


In [37]:
print(y[:3])

[ 269  371 1115]


In [38]:
y = to_categorical(y, num_classes=vocab_size)

In [39]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM

In [40]:
embedding_dim = 10
hidden_units = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)

Epoch 1/200
244/244 - 5s - loss: 7.6404 - accuracy: 0.0314 - 5s/epoch - 20ms/step
Epoch 2/200
244/244 - 3s - loss: 7.1164 - accuracy: 0.0301 - 3s/epoch - 11ms/step
Epoch 3/200
244/244 - 3s - loss: 6.9802 - accuracy: 0.0345 - 3s/epoch - 14ms/step
Epoch 4/200
244/244 - 3s - loss: 6.8486 - accuracy: 0.0401 - 3s/epoch - 12ms/step
Epoch 5/200
244/244 - 3s - loss: 6.6927 - accuracy: 0.0465 - 3s/epoch - 12ms/step
Epoch 6/200
244/244 - 3s - loss: 6.5209 - accuracy: 0.0461 - 3s/epoch - 11ms/step
Epoch 7/200
244/244 - 3s - loss: 6.3335 - accuracy: 0.0529 - 3s/epoch - 12ms/step
Epoch 8/200
244/244 - 3s - loss: 6.1347 - accuracy: 0.0572 - 3s/epoch - 14ms/step
Epoch 9/200
244/244 - 3s - loss: 5.9353 - accuracy: 0.0629 - 3s/epoch - 11ms/step
Epoch 10/200
244/244 - 3s - loss: 5.7474 - accuracy: 0.0642 - 3s/epoch - 12ms/step
Epoch 11/200
244/244 - 3s - loss: 5.5689 - accuracy: 0.0697 - 3s/epoch - 11ms/step
Epoch 12/200
244/244 - 3s - loss: 5.4007 - accuracy: 0.0756 - 3s/epoch - 10ms/step
Epoch 13/200


Epoch 100/200
244/244 - 2s - loss: 0.5864 - accuracy: 0.8808 - 2s/epoch - 10ms/step
Epoch 101/200
244/244 - 3s - loss: 0.5747 - accuracy: 0.8822 - 3s/epoch - 12ms/step
Epoch 102/200
244/244 - 3s - loss: 0.5692 - accuracy: 0.8848 - 3s/epoch - 11ms/step
Epoch 103/200
244/244 - 3s - loss: 0.5576 - accuracy: 0.8874 - 3s/epoch - 11ms/step
Epoch 104/200
244/244 - 3s - loss: 0.5418 - accuracy: 0.8884 - 3s/epoch - 14ms/step
Epoch 105/200
244/244 - 3s - loss: 0.5291 - accuracy: 0.8916 - 3s/epoch - 11ms/step
Epoch 106/200
244/244 - 3s - loss: 0.5191 - accuracy: 0.8926 - 3s/epoch - 12ms/step
Epoch 107/200
244/244 - 3s - loss: 0.5174 - accuracy: 0.8943 - 3s/epoch - 12ms/step
Epoch 108/200
244/244 - 4s - loss: 0.5030 - accuracy: 0.8979 - 4s/epoch - 15ms/step
Epoch 109/200
244/244 - 3s - loss: 0.4926 - accuracy: 0.8973 - 3s/epoch - 13ms/step
Epoch 110/200
244/244 - 3s - loss: 0.4843 - accuracy: 0.8981 - 3s/epoch - 11ms/step
Epoch 111/200
244/244 - 3s - loss: 0.4752 - accuracy: 0.9006 - 3s/epoch - 12

Epoch 198/200
244/244 - 3s - loss: 0.2645 - accuracy: 0.9164 - 3s/epoch - 10ms/step
Epoch 199/200
244/244 - 3s - loss: 0.2708 - accuracy: 0.9149 - 3s/epoch - 10ms/step
Epoch 200/200
244/244 - 3s - loss: 0.2836 - accuracy: 0.9146 - 3s/epoch - 11ms/step


In [41]:
def sentence_generation(model, tokenizer, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word
    sentence = ''

    # n번 반복
    for _ in range(n):
        encoded = tokenizer.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=max_len-1, padding='pre')

        # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
        result = model.predict(encoded, verbose=0)
        result = np.argmax(result, axis=1)

        for word, index in tokenizer.word_index.items(): 
            # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
            if index == result:
                break

        # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        current_word = current_word + ' '  + word

        # 예측 단어를 문장에 저장
        sentence = sentence + ' ' + word

    sentence = init_word + sentence
    return sentence

In [42]:
print(sentence_generation(model, tokenizer, 'i', 10))

i cant jump ship from facebook yet house control in 92


In [43]:
print(sentence_generation(model, tokenizer, 'how', 10))

how to make facebook more accountable is just gluten viral push
